# 1、TextSplitter的使用

1、使用细节

①TextSplitter作为各种具体的文档拆分器的父类

②内部定义了一些常用的属性：

chunk_size：返回块的最大尺寸, 单位是字符数, 默认值为4000（由长度函数测量）

chunk_overlap：相邻两个块之间的字符重叠数, 避免信息在边界处被切断而丢失. 默认值为200, 通常会设置为chunk_size的10%~20%

length_function：用于测量给定块字符数的函数, 默认赋值为len的函数. len函数在Python中按Unicode字符计数, 所以一个汉字、一个英文字母、一个符号都算一个字符。

keep_separator：是否在块中保留分隔符, 默认值为False.

add_start_index：如果为`True`, 则在元数据中包含块的起始索引, 默认值为False

strip_whitespace：如果为`True`, 则从每个文档的开始和结束处去除空白字符, 默认为True

③内部定义了一些常用的方法：

情况1：按照字符串进行切分

split_text(xxx): 传入的参数类型：字符串; 返回值的类型 List[str]

create_documents(xxx): 传入的参数类型：List[str]; 返回值类型：List[Document]; 底层调用了split_text(xxx)

情况2：按照Document对象进行拆分

split_documents(xxx): 传入的参数类型：List[Document]; 返回值的类型：List[Document]; 底层调用了create_documents(xxx)


2、Document对象 与 Str 是什么关系？

文档切分器可以按照字符进行切分, 也可以按照Document进行切分. 其中, Str可以理解为Document对象的page_content属性.

# 2、具体的拆分器①：CharacterTextSplitter：Split by Character

举例1：体会chunk_size、chunk_overlap

说明：若必须禁用分割符（如处理无空格文本, 需容忍实际块长略小于chunk_size（尤其对中文））

In [2]:
# 1.导入相关依赖
from langchain.text_splitter import CharacterTextSplitter

# 2.示例文本
text = """
LangChain 是一个用于开发由语言模型驱动的应用程序的框架的。它提供了一套工具和抽象，使开发者
能够更容易地构建复杂的应用程序。
"""

# 3.定义字符分割器
splitter = CharacterTextSplitter(
    chunk_size=50,  # 每块大小
    chunk_overlap=5,  # 块与块之间的重复字符数
    #length_function=len,
    separator=""  # 设置为空字符串时，表示禁用分隔符优先
)

# 4.分割文本
texts = splitter.split_text(text)

# 5.打印结果
for i, chunk in enumerate(texts):
    print(f"块 {i + 1}:长度：{len(chunk)}")
    print(chunk)
    print("-" * 50)


块 1:长度：49
LangChain 是一个用于开发由语言模型驱动的应用程序的框架的。它提供了一套工具和抽象，使开发
--------------------------------------------------
块 2:长度：23
象，使开发者
能够更容易地构建复杂的应用程序。
--------------------------------------------------


举例2：体会separator

注意：无重叠

separator优先原则：当设置了`separator`（如"。"）, 分割器会首先尝试在分隔符处分割, 然后再考虑chunk_size。这是为了避免再句子中间硬性切断。这种设计是为了：

①优先保持语义完整性（不切断句子）

②避免产生无意义的碎片（如半个单词/不完整句子）

③如果`chunk_size`比片段小, 无法拆分片段, 导致overlap失效

④chunk_overlap仅在合并后的片段之间生效（如果`chunk_size`足够大）。如果没有合并的片段, 则overlap失效

In [4]:
# 1.导入相关依赖
from langchain.text_splitter import CharacterTextSplitter

# 2.定义要分割的文本
text = "这是一个示例文本啊。我们将使用CharacterTextSplitter将其分割成小块。分割基于字符数。"
# text = """
# LangChain 是一个用于开发由语言模型。驱动的应用程序的框架的。它提供了一套工具和抽象。使开发者能够更容易地构建复杂的应用程序。
# """

# 3.定义分割器实例
text_splitter = CharacterTextSplitter(
    chunk_size=30,  # 每个块的最大字符数
    chunk_overlap=5,  # 块之间的重叠字符数
    separator="。",  # 按句号分割（分隔符优先）
)

# 4.开始分割
chunks = text_splitter.split_text(text)

# 5.打印效果
for i, chunk in enumerate(chunks):
    print(f"块 {i + 1}:长度：{len(chunk)}")
    print(chunk)
    print("-" * 50)

Created a chunk of size 33, which is longer than the specified 30


块 1:长度：9
这是一个示例文本啊
--------------------------------------------------
块 2:长度：33
我们将使用CharacterTextSplitter将其分割成小块
--------------------------------------------------
块 3:长度：7
分割基于字符数
--------------------------------------------------


举例3：熟悉 keep_separator、separator 以及 chunk_overlap 何时生效

In [5]:
# 1.导入相关依赖
from langchain.text_splitter import CharacterTextSplitter

# 2.定义要分割的文本
text = "这是第一段文本。这是第二段内容。最后一段结束。"

# 3.定义字符分割器
text_splitter = CharacterTextSplitter(
    separator="。",
    chunk_size=20,
    chunk_overlap=8,
    keep_separator=True  #chunk中是否保留切割符
)

# 4.分割文本
chunks = text_splitter.split_text(text)

# 5.打印结果
for i, chunk in enumerate(chunks):
    print(f"块 {i + 1}:长度：{len(chunk)}")
    print(chunk)
    print("-" * 50)

块 1:长度：15
这是第一段文本。这是第二段内容
--------------------------------------------------
块 2:长度：16
。这是第二段内容。最后一段结束。
--------------------------------------------------


# 3、具体的拆分器②：RecursiveCharacterTextSplitter 最常用

举例1：使用split_text()

In [6]:
# 1、导入相关依赖
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 2、定义RecursiveCharacterTextSplitter分割器对象
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=10,
    chunk_overlap=0,
    add_start_index=True,
)

# 3、定义拆分的内容
text = "LangChain框架特性\n\n多模型集成(GPT/Claude)\n记忆管理功能\n链式调用设计。文档分析场景示例：需要处理PDF/Word等格式。"

# 4、拆分器分割
paragraphs = text_splitter.split_text(text)
for para in paragraphs:
    print(para)
    print('--------')

LangChain框
--------
架特性
--------
多模型集成(GPT
--------
/Claude)
--------
记忆管理功能
--------
链式调用设计。文档
--------
分析场景示例：需要处
--------
理PDF/Word等
--------
格式。
--------


举例2：使用create_documents()方法演示，传入字符串列表，返回Document对象列表

In [2]:
# 1、导入相关依赖
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 2、定义RecursiveCharacterTextSplitter分割器对象
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=10,
    chunk_overlap=0,
    add_start_index=True,  # 打开或者关闭可以控制首字符在源文件中的起始索引
)

# 3、定义分割器的内容
list = [
    "LangChain框架特性\n\n多模型集成(GPT/Claude)\n记忆管理功能\n链式调用设计。文档分析场景示例：需要处理PDF/Word等格式。"]

# 4、分割器分割
paragraphs = text_splitter.create_documents(list)

for paragraph in paragraphs:
    print(paragraph)
    print('---------')

page_content='LangChain框'
---------
page_content='架特性'
---------
page_content='多模型集成(GPT'
---------
page_content='/Claude)'
---------
page_content='记忆管理功能'
---------
page_content='链式调用设计。文档'
---------
page_content='分析场景示例：需要处'
---------
page_content='理PDF/Word等'
---------
page_content='格式。'
---------


举例3：使用create_documents()方法演示，将本地文件内容加载成字符串，进行拆分

In [3]:
# 1.导入相关依赖
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 2.打开.txt文件
with open("asset/load/08-ai.txt", encoding="utf-8") as f:
    state_of_the_union = f.read()  #返回的是字符串

# 3.定义RecursiveCharacterTextSplitter（递归字符分割器）
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
    #chunk_overlap=0,
    length_function=len
)

# 4.分割文本
texts = text_splitter.create_documents([state_of_the_union])

# 5.打印分割文本
for text in texts:
    print(f"🔥{text.page_content}")

🔥人工智能（AI）是什么？
🔥人工智能（Artificial
🔥Intelligence，简称AI）是指由计算机系统模拟人类智能的技术，使其能够执行通常需要人类认知能力的任务，如学习、推理、决策和语言理解。AI的核心目标是让机器具备感知环境、处理信息并自主行动的
🔥让机器具备感知环境、处理信息并自主行动的能力。
🔥1. AI的技术基础
AI依赖多种关键技术：

机器学习（ML）：通过算法让计算机从数据中学习规律，无需显式编程。例如，推荐系统通过用户历史行为预测偏好。
🔥深度学习：基于神经网络的机器学习分支，擅长处理图像、语音等复杂数据。AlphaGo击败围棋冠军便是典型案例。

自然语言处理（NLP）：使计算机理解、生成人类语言，如ChatGPT的对话能力。
🔥2. AI的应用场景
AI已渗透到日常生活和各行各业：

医疗：辅助诊断（如AI分析医学影像）、药物研发加速。

交通：自动驾驶汽车通过传感器和AI算法实现安全导航。
🔥金融：欺诈检测、智能投顾（如风险评估模型）。

教育：个性化学习平台根据学生表现调整教学内容。

3. AI的挑战与未来
尽管前景广阔，AI仍面临问题：
🔥伦理争议：数据隐私、算法偏见（如招聘AI歧视特定群体）。

就业影响：自动化可能取代部分人工岗位，但也会创造新职业。

技术瓶颈：通用人工智能（AGI）尚未实现，当前AI仅擅长特定任务。
🔥未来，AI将与人类协作而非替代：医生借助AI提高诊断效率，教师利用AI定制课程。其发展需平衡技术创新与社会责任，确保技术造福全人类。


举例4：使用split_documents()方法演示，利用PDFLoader加载文档，对文档的内容用递归切割器切割

In [4]:
# 1.导入相关依赖
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 2.定义PyPDFLoader加载器
loader = PyPDFLoader("./asset/load/02-load.pdf")

# 3.加载和切割文档对象
docs = loader.load()  # 返回Document对象构成的list
# print(f"第0页：\n{docs[0]}")

# 4.定义切割器
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    #chunk_size=120,
    chunk_overlap=0,
    # chunk_overlap=100,
    length_function=len,
    add_start_index=True,
)

# 5.对pdf内容进行切割得到文档对象
paragraphs = text_splitter.split_documents(docs)
#paragraphs = text_splitter.create_documents([text])

for para in paragraphs:
    print(para.page_content)
    print('-------')


"他的车，他的命！ 他忽然想起来，一年，二年，至少有三四年；一滴汗，两滴汗，不
知道多少万滴汗，才挣出那辆车。从风里雨里的咬牙，从饭里茶里的自苦，才赚出那辆车。
那辆车是他的一切挣扎与困苦的总结果与报酬，像身经百战的武士的一颗徽章。……他老想
着远远的一辆车，可以使他自由，独立，像自己的手脚的那么一辆车。" 
 
"他吃，他喝，他嫖，他赌，他懒，他狡猾， 因为他没了心，他的心被人家摘了去。他
-------
只剩下那个高大的肉架子，等着溃烂，预备着到乱死岗子去。……体面的、要强的、好梦想
的、利己的、个人的、健壮的、伟大的祥子，不知陪着人家送了多少回殡；不知道何时何地
会埋起他自己来， 埋起这堕落的、 自私的、 不幸的、 社会病胎里的产儿， 个人主义的末路鬼！
"
-------
